# MÔ HÌNH DỰ BÁO NGUY CƠ NGẬP LỤT SỬ DỤNG RANDOM FOREST

Notebook này thực hiện các bước:
1. Đọc dữ liệu điểm huấn luyện từ file CSV
2. Trích xuất 13 đặc trưng từ ảnh vệ tinh
3. Huấn luyện mô hình Random Forest
4. Dự đoán nguy cơ ngập lụt trên khu vực nghiên cứu

In [ ]:
# Cài đặt các thư viện cần thiết
!pip install geopandas rasterio scikit-learn pandas numpy

## BƯỚC 1: NHẬP CÁC THƯ VIỆN CẦN THIẾT

Trong bước này, chúng ta nhập các thư viện cần thiết để xử lý dữ liệu không gian và huấn luyện mô hình:
- `geopandas`: Xử lý dữ liệu địa lý
- `rasterio`: Đọc và xử lý ảnh vệ tinh
- `scikit-learn`: Huấn luyện mô hình máy học
- `pandas`, `numpy`: Xử lý dữ liệu dạng bảng

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

## BƯỚC 2: ĐỌC DỮ LIỆU ĐIỂM HUẤN LUYỆN

Đọc file CSV chứa tọa độ (lat, lon) và nhãn flood (0 hoặc 1) để làm dữ liệu huấn luyện.

In [ ]:
# Đọc dữ liệu điểm huấn luyện từ file CSV
# Giả sử file có tên 'training_points.csv' với 3 cột: lat, lon, flood
df_points = pd.read_csv('training_points.csv')

print("Thông tin dữ liệu điểm huấn luyện:")
print(f"Số lượng điểm: {len(df_points)}")
print(f"Các cột: {list(df_points.columns)}")
print(f"Phân bố nhãn flood:")
print(df_points['flood'].value_counts())

# Hiển thị 5 dòng đầu tiên
df_points.head()

## BƯỚC 3: TẢI 13 ẢNH ĐẶC TRƯNG

Tải 13 ảnh vệ tinh tương ứng với 13 đặc trưng:
1. lulc - Loại sử dụng đất
2. Density_River - Mật độ sông
3. Density_Road - Mật độ đường
4. Distan2river - Khoảng cách đến sông
5. Distan2road_met - Khoảng cách đến đường (mét)
6. aspect - Hướng dốc
7. curvature - Độ cong
8. dem - Độ cao (DEM)
9. flowDir - Hướng dòng chảy
10. slope - Độ dốc
11. twi - Chỉ số độ ẩm địa hình
12. NDVI - Chỉ số thực vật
13. rainfall - Lượng mưa

In [ ]:
# Danh sách tên các đặc trưng
feature_names = [
    'lulc', 'Density_River', 'Density_Road', 'Distan2river', 'Distan2road_met',
    'aspect', 'curvature', 'dem', 'flowDir', 'slope', 'twi', 'NDVI', 'rainfall'
]

# Giả sử các ảnh được lưu trong thư mục 'features' với tên tương ứng
feature_files = {
    'lulc': 'features/lulc.tif',
    'Density_River': 'features/Density_River.tif',
    'Density_Road': 'features/Density_Road.tif',
    'Distan2river': 'features/Distan2river.tif',
    'Distan2road_met': 'features/Distan2road_met.tif',
    'aspect': 'features/aspect.tif',
    'curvature': 'features/curvature.tif',
    'dem': 'features/dem.tif',
    'flowDir': 'features/flowDir.tif',
    'slope': 'features/slope.tif',
    'twi': 'features/twi.tif',
    'NDVI': 'features/NDVI.tif',
    'rainfall': 'features/rainfall.tif'
}

# Mở thử một ảnh để kiểm tra
print("Kiểm tra thông tin ảnh đặc trưng:")
with rasterio.open(feature_files['dem']) as src:
    print(f"Kích thước ảnh DEM: {src.height} x {src.width}")
    print(f"Hệ tọa độ: {src.crs}")
    print(f"Giới hạn tọa độ: {src.bounds}")

## BƯỚC 4: TRÍCH XUẤT ĐẶC TRƯNG TỪ ẢNH

Với mỗi điểm tọa độ trong dữ liệu huấn luyện, trích xuất giá trị của 13 đặc trưng từ các ảnh tương ứng.

In [ ]:
def extract_features_from_images(df_points, feature_files, feature_names):
    """
    Trích xuất giá trị đặc trưng từ ảnh vệ tinh cho từng điểm
    """
    # Tạo bản sao của dataframe điểm
    df_features = df_points.copy()
    
    # Duyệt qua từng đặc trưng
    for feature_name in feature_names:
        feature_file = feature_files[feature_name]
        print(f"Đang trích xuất đặc trưng: {feature_name}...")
        
        # Mở ảnh đặc trưng
        with rasterio.open(feature_file) as src:
            # Trích xuất giá trị tại từng tọa độ
            values = []
            for idx, row in df_points.iterrows():
                # Chuyển tọa độ lat/lon sang tọa độ ảnh
                x, y = row['lon'], row['lat']
                
                # Trích xuất giá trị tại tọa độ này
                try:
                    # Sử dụng sample để lấy giá trị tại tọa độ cụ thể
                    for val in src.sample([(x, y)]):
                        values.append(val[0] if val[0] is not None else np.nan)
                except:
                    values.append(np.nan)
            
            # Thêm cột đặc trưng vào dataframe
            df_features[feature_name] = values
    
    return df_features

# Trích xuất đặc trưng cho tất cả các điểm
df_training = extract_features_from_images(df_points, feature_files, feature_names)

print("\nDữ liệu sau khi trích xuất đặc trưng:")
print(f"Số cột: {len(df_training.columns)}")
print(f"Tên các cột: {list(df_training.columns)}")

# Hiển thị một số dòng
df_training.head()

## BƯỚC 5: TIỀN XỬ LÝ DỮ LIỆU

Loại bỏ các điểm có giá trị đặc trưng bị thiếu và chuẩn bị dữ liệu cho huấn luyện.

In [ ]:
# Kiểm tra giá trị thiếu
print("Số lượng giá trị thiếu trong từng cột:")
missing_values = df_training.isnull().sum()
print(missing_values[missing_values > 0])

# Loại bỏ các điểm có giá trị thiếu
df_clean = df_training.dropna()
print(f"\nSố điểm trước khi loại bỏ giá trị thiếu: {len(df_training)}")
print(f"Số điểm sau khi loại bỏ giá trị thiếu: {len(df_clean)}")

# Tách đặc trưng và nhãn
X = df_clean[feature_names]
y = df_clean['flood']

print(f"\nKích thước dữ liệu huấn luyện:")
print(f"Đặc trưng (X): {X.shape}")
print(f"Nhãn (y): {y.shape}")

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"\nChia dữ liệu:")
print(f"Tập huấn luyện: {X_train.shape[0]} điểm")
print(f"Tập kiểm tra: {X_test.shape[0]} điểm")

## BƯỚC 6: HUẤN LUYỆN MÔ HÌNH RANDOM FOREST

Sử dụng Random Forest Regressor để huấn luyện mô hình dự đoán nguy cơ ngập lụt.

In [ ]:
# Khởi tạo mô hình Random Forest với các tham số đã tối ưu
rf_model = RandomForestRegressor(
    n_estimators=100,        # Số cây trong rừng
    max_depth=20,            # Độ sâu tối đa của cây
    min_samples_split=10,    # Số mẫu tối thiểu để chia nút
    min_samples_leaf=5,      # Số mẫu tối thiểu trong lá
    random_state=42,         # Để kết quả có thể tái tạo
    n_jobs=-1                # Sử dụng tất cả các lõi CPU
)

print("Đang huấn luyện mô hình Random Forest...")
# Huấn luyện mô hình
rf_model.fit(X_train, y_train)
print("Hoàn thành huấn luyện mô hình!")

# Dự đoán trên tập kiểm tra
y_pred = rf_model.predict(X_test)

# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nKết quả đánh giá mô hình:")
print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")

# Hiển thị tầm quan trọng của các đặc trưng
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nTầm quan trọng của các đặc trưng:")
print(feature_importance)

## BƯỚC 7: DỰ ĐOÁN TRÊN KHU VỰC NGHIÊN CỨU

Sử dụng mô hình đã huấn luyện để dự đoán nguy cơ ngập lụt trên toàn bộ khu vực nghiên cứu.

In [ ]:
# Đọc khu vực nghiên cứu từ file shapefile
study_area = gpd.read_file('study_area.shp')
print(f"Thông tin khu vực nghiên cứu:")
print(f"Hệ tọa độ: {study_area.crs}")
print(f"Số lượng đối tượng: {len(study_area)}")

# Tạo lưới điểm trên khu vực nghiên cứu để dự đoán
def create_prediction_grid(study_area, resolution=100):
    """
    Tạo lưới điểm đều trên khu vực nghiên cứu để dự đoán
    """
    # Lấy giới hạn của khu vực nghiên cứu
    bounds = study_area.total_bounds  # [minx, miny, maxx, maxy]
    
    # Tạo lưới điểm
    x_coords = np.arange(bounds[0], bounds[2], resolution)
    y_coords = np.arange(bounds[1], bounds[3], resolution)
    
    # Tạo tất cả các tổ hợp tọa độ
    points = []
    for x in x_coords:
        for y in y_coords:
            points.append((x, y))
    
    # Tạo GeoDataFrame
    grid_gdf = gpd.GeoDataFrame(
        {'id': range(len(points))},
        geometry=[gpd.points_from_xy([p[0]], [p[1]])[0] for p in points],
        crs=study_area.crs
    )
    
    # Chỉ giữ lại các điểm nằm trong khu vực nghiên cứu
    grid_gdf = gpd.sjoin(grid_gdf, study_area, how='inner', op='within')
    
    return grid_gdf

# Tạo lưới điểm dự đoán
print("\nĐang tạo lưới điểm dự đoán...")
prediction_grid = create_prediction_grid(study_area, resolution=30)  # 30m resolution
print(f"Số điểm để dự đoán: {len(prediction_grid)}")

# Trích xuất đặc trưng cho các điểm dự đoán
print("\nĐang trích xuất đặc trưng cho các điểm dự đoán...")
df_prediction_points = pd.DataFrame({
    'lat': prediction_grid.geometry.y,
    'lon': prediction_grid.geometry.x
})

df_prediction_features = extract_features_from_images(df_prediction_points, feature_files, feature_names)
df_prediction_clean = df_prediction_features.dropna()

print(f"Số điểm hợp lệ sau khi trích xuất đặc trưng: {len(df_prediction_clean)}")

# Dự đoán nguy cơ ngập lụt
print("\nĐang dự đoán nguy cơ ngập lụt...")
X_predict = df_prediction_clean[feature_names]
flood_probabilities = rf_model.predict(X_predict)

print(f"Dự đoán hoàn tất!")
print(f"Phạm vi xác suất ngập: {flood_probabilities.min():.4f} - {flood_probabilities.max():.4f}")

# Thêm kết quả dự đoán vào dataframe
df_prediction_clean['flood_probability'] = flood_probabilities

# Hiển thị một số kết quả
print("\nMột số kết quả dự đoán:")
df_prediction_clean[['lat', 'lon', 'flood_probability']].head(10)

## BƯỚC 8: XUẤT KẾT QUẢ

Lưu kết quả dự đoán vào file CSV để sử dụng sau này.

In [ ]:
# Lưu kết quả dự đoán vào file CSV
output_file = 'flood_prediction_results.csv'
df_prediction_clean[['lat', 'lon', 'flood_probability']].to_csv(output_file, index=False)
print(f"Kết quả đã được lưu vào file: {output_file}")

# Lưu mô hình để sử dụng sau này
import joblib
model_file = 'flood_prediction_model.pkl'
joblib.dump(rf_model, model_file)
print(f"Mô hình đã được lưu vào file: {model_file}")

print("\n===== HOÀN THÀNH =====")
print("Mô hình dự báo nguy cơ ngập lụt đã được huấn luyện và chạy dự đoán thành công!")